In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, roc_curve
from sklearn.model_selection import train_test_split

# Load the Amazon dataset
print('PCA Dataset')
amazon = pd.read_csv('/kaggle/input/amazon-kaggle-with-pca/Amazon_Kaggle_with_PCA.csv')

# Load the output data
output = pd.read_csv('/kaggle/input/train-test-amazon1/train_test_amazon1.sample', delimiter=' ')

# Extract the last column from the output DataFrame
amazon = amazon[1:]
output = output.iloc[:, -1]

print("Done")


In [ ]:
from sklearn.svm import SVC

from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, precision_recall_curve, auc

x_train, x_test, y_train, y_test = train_test_split(amazon,output,random_state=1)


param_dist = {'C': uniform(0.1, 100)}

#clf = SVC(kernel='rbf', verbose=True)
# Perform random search
#random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=10, cv=5)
#random_search.fit(x_train, y_train)
#print(x_train.shape)
#print(x_train)
#print(y_train.shape)
#print(y_train)
# Best C value
#best_C = random_search.best_params_['C']
#print("best C: ", best_C)
#81.59


clf = SVC(C=81.59, kernel='rbf', verbose=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
#y_pred = to_categorical(y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
# Calculate accuracy and F1 score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
fpr, tpr, threshold = roc_curve(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision Score:", precision)
print("Recall Score:", recall)
print("FPR Score:", fpr)
print("TPR Score", tpr)

print (y_pred)
print (y_test)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn import metrics

# Assuming clf is your trained classifier and x_test is your test data
y_pred = clf.predict(x_test)

# Convert to NumPy array if y_test is a Pandas Series and flatten if necessary
y_test_flat = y_test.to_numpy().flatten()

# Flatten the predicted values (if needed)
y_pred_flat = y_pred.flatten()

# Convert to binary classification (example)
# Adjust this step according to your specific problem
threshold = 0.5
y_pred_binary = (y_pred_flat > threshold).astype(int)

# Reshape y_test_flat to match the shape of y_pred_binary
y_test_binary = (y_test_flat[:len(y_pred_binary)] > threshold).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test_binary, y_pred_binary)
f1 = f1_score(y_test_binary, y_pred_binary, average='weighted')
precision = precision_score(y_test_binary, y_pred_binary)
recall = recall_score(y_test_binary, y_pred_binary)

# Confusion matrix
cm = confusion_matrix(y_test_binary, y_pred_binary)

# True Positive Rate (TPR) and False Positive Rate (FPR)
tpr_value = cm[1, 1] / (cm[1, 1] + cm[1, 0])
fpr_value = cm[0, 1] / (cm[0, 1] + cm[0, 0])

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision Score:", precision)
print("Recall Score:", recall)
print("TPR Score:", tpr_value)
print("FPR Score:", fpr_value)


In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc

precision_a, recall_a, threshold = precision_recall_curve(y_test_binary, y_pred_binary)
prc_auc = auc(recall_a, precision_a)
print("Area Under the PR Curve score: ", prc_auc)


fpr, tpr, threshold = roc_curve(y_test_binary, y_pred_binary)
roc_auc = auc(fpr, tpr)
print("Area Under the ROC Curve score: ", roc_auc)

In [ ]:
import json

# Assuming tpr and fpr are NumPy arrays
print("Type of f1:", type(f1))
print("Type of precision:", type(precision))
print("Type of recall:", type(recall))
print("Type of tpr_value:", type(tpr_value))
print("Type of fpr_value:", type(fpr_value))
print("Type of roc_auc:", type(roc_auc))
print("Type of prc_auc:", type(prc_auc))
print("Type of tpr:", type(tpr))
print("Type of fpr:", type(fpr))

precision_list = precision_a.tolist()
recall_list = recall_a.tolist()
tpr_list = tpr.tolist()
fpr_list = fpr.tolist()

#print(f1.type())

data = {
    "Average F1 Score": f1, 
    "Average Precision": precision, 
    "Average Recall": recall, 
    "Average True Positive Rate": tpr_value, 
    "Average False Positive Rate": fpr_value, 
    "Average ROC AUC": roc_auc, 
    "Average PRC AUC": prc_auc,
    "TPR Array" : tpr_list,
    "FPR Array" : fpr_list,
    "Precision Array": precision_list, 
    "Recall Array": recall_list
}

# Specify the file path
file_path = "/kaggle/working/SVM - Amazon Kaggle with PCA - No S5KCV.json"

# Write the data to a JSON file
with open(file_path, "w") as json_file:
    json.dump(data, json_file)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import SVC
import numpy as np

# Assuming amazon contains your features and output contains your labels
X = amazon
y = output

# Define number of folds
k = 5

kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)

f1_list = []
accuracies = []
precision_l = []
recall_l = []

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    clfk2 = SVC(random_state=0)
    clfk2.fit(X_train, y_train)
    y_pred = clfk2.predict(X_test)
    
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    
    f1_list.append(f1)
    accuracies.append(accuracy)
    precision_l.append(precision)
    recall_l.append(recall)

avg_f1 = np.mean(f1_list)
avg_accuracy = np.mean(accuracies)
avg_precision = np.mean(precision_l)
avg_recall = np.mean(recall_l)

print("Average F1 Score:", avg_f1)
print("Average Accuracy:", avg_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn import metrics

y_pred = clfk2.predict(x_test)

# Flatten the predicted and actual values
y_pred_flat = y_pred.flatten()
y_test_flat = y_test.to_numpy().flatten()

# Convert to binary classification (example)
# Adjust this step according to your specific problem
threshold = 0.5
y_pred_binary = (y_pred_flat[:6553] > threshold).astype(int)

# Reshape y_test_flat to match the shape of y_pred_flat
y_test_binary = (y_test_flat[:len(y_pred_binary)] > threshold).astype(int)
#y_pred = to_categorical(y_pred)
f1 = f1_score(y_test_binary, y_pred_binary, average='weighted')
# Calculate accuracy and F1 score
accuracy = accuracy_score(y_test_binary, y_pred_binary)
precision = precision_score(y_test_binary, y_pred_binary)
recall = recall_score(y_test_binary, y_pred_binary)

cm = confusion_matrix(y_test_binary.ravel(), y_test_binary.ravel())
#np.savetxt('cm.txt', cm, delimiter=',', fmt='%f')
#f1 = f1_score(y_test.ravel(), y_pred.ravel())
tpr_value = cm[1, 1] / (cm[1, 1] + cm[1, 0])
fpr_value = cm[0, 1] / (cm[0, 1] + cm[0, 0])

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision Score:", precision)
print("Recall Score:", recall)
print("TPR Score:", tpr_value)
print("FPR Score:", fpr_value)


In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc

precision_a, recall_a, threshold = precision_recall_curve(y_test_binary, y_pred_binary)
prc_auc = auc(recall_a, precision_a)
print("Area Under the PR Curve score: ", prc_auc)


fpr, tpr, threshold = roc_curve(y_test_binary, y_pred_binary)
roc_auc = auc(fpr, tpr)
print("Area Under the ROC Curve score: ", roc_auc)

In [ ]:
import json

# Assuming tpr and fpr are NumPy arrays
print("Type of f1:", type(f1))
print("Type of precision:", type(precision))
print("Type of recall:", type(recall))
print("Type of tpr_value:", type(tpr_value))
print("Type of fpr_value:", type(fpr_value))
print("Type of roc_auc:", type(roc_auc))
print("Type of prc_auc:", type(prc_auc))
print("Type of tpr:", type(tpr))
print("Type of fpr:", type(fpr))

precision_list = precision_a.tolist()
recall_list = recall_a.tolist()
tpr_list = tpr.tolist()
fpr_list = fpr.tolist()

#print(f1.type())

data = {
    "Average F1 Score": f1, 
    "Average Precision": precision, 
    "Average Recall": recall, 
    "Average True Positive Rate": tpr_value, 
    "Average False Positive Rate": fpr_value, 
    "Average ROC AUC": roc_auc, 
    "Average PRC AUC": prc_auc,
    "TPR Array" : tpr_list,
    "FPR Array" : fpr_list,
    "Precision Array": precision_list, 
    "Recall Array": recall_list
}

# Specify the file path
file_path = "/kaggle/working/SVM - Amazon Kaggle with PCA - SKCV.json"

# Write the data to a JSON file
with open(file_path, "w") as json_file:
    json.dump(data, json_file)